In [1]:
import warnings; warnings.simplefilter('ignore')
try:
    import google.colab
    IN_COLAB = True
    !mkdir data models images
    #!wget -P data/ https://raw.githubusercontent.com/taudataanalytics/Pedas-2025/master/data/Tech_stocks.csv
    #!pip install darts nolds tsfresh seaborn prophet plotly ipywidgets --q
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded


In [2]:
import darts, numpy as np, pandas as pd, seaborn as sns
import matplotlib, matplotlib.pyplot as plt
from darts.metrics import smape, mape

seed = 42
np.random.seed(seed)
"Done"

'Done'